---
title: Tensorflow2 Model Input 形状指定
tags: 小书匠,tensorflow2,model,input,input_shape
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# Tensorflow2 Model Input 形状指定

In [10]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.4.1


## input 是 dict 时形状的指定

In [2]:
class Model(tf.keras.Model):

    def __init__(self):
        super().__init__()
        self._movie_full_dense = tf.keras.layers.Dense(
            units=40, activation=tf.keras.layers.Activation("relu"))
        self._user_full_dense = tf.keras.layers.Dense(
            units=40, activation=tf.keras.layers.Activation("relu"))
        self._full_dense = tf.keras.layers.Dense(
            units=1, activation=tf.keras.layers.Activation("sigmoid"))

    def call(self, features):
        movie_embedding = self._movie_full_dense(features['movie'])
        user_embedding = self._user_full_dense(features['user'])
        combined = tf.concat([movie_embedding, user_embedding], 1)
        output = self._full_dense(combined)
        return output

### 方法一：在 `tf.keras.Input` 中通过 name 指定

In [12]:
import tensorflow as tf

print(tf.version.VERSION)

toy_data = {
    'user': [[10], [12], [12], [10]],
    'movie': [[0], [1], [0], [1]],
}

# dataset 中每一个 batch 是一个 dict
dataset = tf.data.Dataset.from_tensor_slices(toy_data).batch(2)


def make_model():
    inp_movie = tf.keras.Input(shape=(1,), name="movie") # 注意，这里的 name 要和 data 中的 key 对应
    inp_user = tf.keras.Input(shape=(1,), name="user")
    movie_embedding = tf.keras.layers.Dense(
        units=40, activation=tf.keras.layers.Activation("relu"))(inp_movie)
    user_embedding = tf.keras.layers.Dense(
        units=40, activation=tf.keras.layers.Activation("relu"))(inp_user)
    combined = tf.concat([movie_embedding, user_embedding], 1)
    output = tf.keras.layers.Dense(
        units=1, activation=tf.keras.layers.Activation("sigmoid"))(combined)
    model = tf.keras.Model(inputs=[inp_movie, inp_user], outputs=output)
    return model

model = make_model()

for x in dataset:
    print(model(x))

2.4.1
tf.Tensor(
[[0.50687456]
 [0.50651133]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.50824934]
 [0.50513655]], shape=(2, 1), dtype=float32)


### 方法二：在 tf.keras.Model 的 inputs 参数中指定

In [13]:
import tensorflow as tf

print(tf.version.VERSION)

toy_data = {
    'user': [[10], [12], [12], [10]],
    'movie': [[0], [1], [0], [1]],
}

dataset = tf.data.Dataset.from_tensor_slices(toy_data).batch(2)

def make_model():
    inp_movie = tf.keras.Input(shape=(1,))
    inp_user = tf.keras.Input(shape=(1,))
    movie_embedding = tf.keras.layers.Dense(
        units=40, activation=tf.keras.layers.Activation("relu"))(inp_movie)
    user_embedding = tf.keras.layers.Dense(
        units=40, activation=tf.keras.layers.Activation("relu"))(inp_user)
    combined = tf.concat([movie_embedding, user_embedding], 1)
    output = tf.keras.layers.Dense(
        units=1, activation=tf.keras.layers.Activation("sigmoid"))(combined)
    model = tf.keras.Model(inputs={"movie": inp_movie, "user": inp_user}, outputs=output)
    return model


model = make_model()

for x in dataset:
    print(model(x))

2.4.1
tf.Tensor(
[[0.933691 ]
 [0.9588034]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.95983577]
 [0.93203425]], shape=(2, 1), dtype=float32)


# References
- http://localhost:8888/lab/tree/DL-Project/learnTensorflow/Tensorflow2%20Model.ipynb
- [Dictionary of tensors input for Keras Functional API TensorFlow 2.0 - Stack Overflow](https://stackoverflow.com/questions/58209678/dictionary-of-tensors-input-for-keras-functional-api-tensorflow-2-0)